In [2]:
pip install skyfield

                                              0.0/453.9 kB ? eta -:--:--
     ------------                           153.6/453.9 kB 4.5 MB/s eta 0:00:01
     -------------------                    235.5/453.9 kB 3.6 MB/s eta 0:00:01
     -------------------------------------  450.6/453.9 kB 4.7 MB/s eta 0:00:01
     -------------------------------------- 453.9/453.9 kB 4.0 MB/s eta 0:00:00
                                              0.0/47.2 kB ? eta -:--:--
     ---------------------------------------- 47.2/47.2 kB ? eta 0:00:00
                                              0.0/160.8 kB ? eta -:--:--
     ---------------------------------------- 160.8/160.8 kB ? eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import skyfield
import re

from IPython.display import clear_output
from skyfield.api import EarthSatellite, utc, wgs84, load
from datetime import datetime, timedelta

In [2]:
logDir = 'L:/DATA/ISIS/ISIS_Test_Metadata_Analysis/MERGED/ISIS-1_MD_Version6.csv'

In [3]:
data = pd.read_csv('L:/DATA/ISIS/Orbitcheck/isis_1_tle.csv')
data.head()

,CCSDS_OMM_VERS,COMMENT,CREATION_DATE,ORIGINATOR,OBJECT_NAME,OBJECT_ID,CENTER_NAME,REF_FRAME,TIME_SYSTEM,MEAN_ELEMENT_THEORY,...,RCS_SIZE,COUNTRY_CODE,LAUNCH_DATE,SITE,DECAY_DATE,FILE,GP_ID,TLE_LINE0,TLE_LINE1,TLE_LINE2
0,2.0,GENERATED VIA SPACE-TRACK.ORG API,2004-08-15T23:36:44,18 SPCS,ISIS 1,1969-009A,EARTH,TEME,UTC,SGP4,...,LARGE,CA,1969-01-30,AFWTR,NaN,33933,8824742,0 ISIS 1,1 03669U 69009 A 69030.87308251 .03349715 +0...,2 03669 088.4203 291.2548 1741532 168.8956 195...
1,2.0,GENERATED VIA SPACE-TRACK.ORG API,2004-08-15T23:36:44,18 SPCS,ISIS 1,1969-009A,EARTH,TEME,UTC,SGP4,...,LARGE,CA,1969-01-30,AFWTR,NaN,33933,8824743,0 ISIS 1,1 03669U 69009 A 69030.87356624 .00016383 +0...,2 03669 088.4174 291.2300 1745981 169.1540 195...
2,2.0,GENERATED VIA SPACE-TRACK.ORG API,2004-08-15T23:36:44,18 SPCS,ISIS 1,1969-009A,EARTH,TEME,UTC,SGP4,...,LARGE,CA,1969-01-30,AFWTR,NaN,33933,8824744,0 ISIS 1,1 03669U 69009 A 69031.58723781 .00000000 +0...,2 03669 088.4297 291.3085 1735280 167.3030 197...
3,2.0,GENERATED VIA SPACE-TRACK.ORG API,2004-08-15T23:36:44,18 SPCS,ISIS 1,1969-009A,EARTH,TEME,UTC,SGP4,...,LARGE,CA,1969-01-30,AFWTR,NaN,33933,8824745,0 ISIS 1,1 03669U 69009 A 69032.12233176 .00000000 +0...,2 03669 088.4154 291.1307 1747976 166.7720 198...
4,2.0,GENERATED VIA SPACE-TRACK.ORG API,2004-08-15T23:36:44,18 SPCS,ISIS 1,1969-009A,EARTH,TEME,UTC,SGP4,...,LARGE,CA,1969-01-30,AFWTR,NaN,33933,8824746,0 ISIS 1,1 03669U 69009 A 69035.15507440 .00000000 +0...,2 03669 088.4244 290.7857 1748660 160.7159 206...


In [4]:
tle = data.loc[:, ["EPOCH", "TLE_LINE1", "TLE_LINE2"]]
tle.EPOCH = pd.to_datetime(tle.EPOCH)

In [5]:
ISIS1 = pd.read_csv(logDir)
ISIS1.head()

,Outlier,Fixed_Frequency_Code,Year,Day,Hour,Minute,Second,Filename,Directory,Subdirectory,...,Adjusted_Satellite_Code,Adjusted_Station_Number,Station_quality,Station_Location_y,Station_ID_y,Number,Station_Latitude,Station_Longitude,Latitude,Longitude
0,NaN,4,1971,61,16,50,30,Image0002.png,b10_R014207713,B1-34-61 ISIS A C-918,...,3,70,1,"Kashima, Japan",KER,70,35.57,140.39,35.6N,140.4E
1,NaN,4,1971,61,16,50,59,Image0003.png,b10_R014207713,B1-34-61 ISIS A C-918,...,3,70,1,"Kashima, Japan",KER,70,35.57,140.39,35.6N,140.4E
2,NaN,4,1971,61,16,51,28,Image0004.png,b10_R014207713,B1-34-61 ISIS A C-918,...,3,70,1,"Kashima, Japan",KER,70,35.57,140.39,35.6N,140.4E
3,NaN,4,1971,61,16,51,57,Image0005.png,b10_R014207713,B1-34-61 ISIS A C-918,...,3,70,1,"Kashima, Japan",KER,70,35.57,140.39,35.6N,140.4E
4,NaN,4,1971,61,16,52,26,Image0006.png,b10_R014207713,B1-34-61 ISIS A C-918,...,3,70,1,"Kashima, Japan",KER,70,35.57,140.39,35.6N,140.4E


In [27]:
# function for settling quadrant ambiguity (before realized skyfield has NWES values available)

def quad_latlon(coords):
    splt_str = re.split(r'(\d+\.?\d*)', coords)
    new_coord = float(''.join(splt_str[1:-1]))
    
    quad = splt_str[-1:]
    
    if quad == ["N"]:
        return new_coord
    elif quad == ["S"]:
        return new_coord*-1
    elif quad == ["E"]:
        return new_coord
    elif quad == ["W"]:
        return new_coord*-1
    else:
        print(splt_str)
        print("An Error has Occured")
        return 0

In [7]:
def nearest(items, pivot):
    return min(items, key=lambda x: abs(x - pivot))

In [32]:
result_dir = []
result_sub_dir = []
result_name = []

result_type = []

result_read_epoch = []
result_read_station = [] 
result_tle_epoch = []

result_flag = []

result_events = []
result_event_times = []

result_alt = []
result_distance =[] 

result_lat = []
result_lon = []
result_height = []

## len(ISIS1)

for i in range(len(ISIS1)):
    record_epoch = ISIS1.loc[i, "Timestamp"]
    try: 
        record_epoch = datetime.strptime(record_epoch, '%Y.%m.%d %H:%M:%S')
    except: 
        record_epoch = datetime(1969, 1, 30, 0, 0, 0)
    station = ISIS1.iloc[i, -2:]
    quality = ISIS1.loc[i, "Time_quality"]
    
    if quality > 3:
        result_dir.append(ISIS1.loc[i, "Directory"])
        result_sub_dir.append(ISIS1.loc[i, "Subdirectory"])
        result_name.append(ISIS1.loc[i, "Filename"])

        result_read_epoch.append(record_epoch)
        result_read_station.append(ISIS1.loc[i, "Station_Location_y"])
        result_tle_epoch.append(0)

        result_flag.append(1)

        result_events.append(0)
        result_event_times.append(0)

        result_alt.append(0)
        result_distance.append(0)

        result_lat.append(0)
        result_lon.append(0)
        result_height.append(0)
        continue 

    nearest_epoch = nearest(tle.EPOCH, record_epoch)
    nearest_tle = tle.loc[tle['EPOCH'] == nearest_epoch]
    
    station = wgs84.latlon(quad_latlon(station.iloc[0]), quad_latlon(station.iloc[1]))
    
    ts = load.timescale()
    satellite = EarthSatellite(nearest_tle.iat[0,1], nearest_tle.iat[0,2], 'ISIS-1', ts)
    
    d = record_epoch.replace(tzinfo=utc)
    t = ts.utc(d)
    
    geocentric = satellite.at(t)
    lat, lon = wgs84.latlon_of(geocentric)
    height = wgs84.height_of(geocentric)
    
#   difference between station
    difference = satellite - station

    topocentric = difference.at(t)
    alt, az, distance = topocentric.altaz()
    
    if alt.degrees > 0:
        flag = 0 
    else:
        flag = 1
    
    if quality == 1:
        t0 = t - 0.010416666666666666
        t1 = t + 0.010416666666666666
    elif quality == 2: 
        continue 
        t0 = t - 0.020833333333333332
        t1 = t + 0.020833333333333332
    elif quality == 3:
        t0 = t
        t1 = t + 0.041667
        
    event_time, events = satellite.find_events(station, t0, t1, altitude_degrees=15.0)
    event_time = event_time.utc_strftime('%Y %b %d %H:%M')

    result_dir.append(ISIS1.loc[i, "Directory"])
    result_sub_dir.append(ISIS1.loc[i, "Subdirectory"])
    result_name.append(ISIS1.loc[i, "Filename"])

    result_read_epoch.append(record_epoch)
    result_read_station.append(ISIS1.loc[i, "Station_Location_y"])
    result_tle_epoch.append(nearest_epoch)

    result_flag.append(flag)

    result_events.append(events)
    result_event_times.append(event_time)

    result_alt.append(alt.degrees)
    result_distance.append(distance.km)

    result_lat.append(lat.degrees)
    result_lon.append(lon.degrees)
    result_height.append(height.km)

    clear_output(wait=True)
    print(i)
    
print("Procsesing Complete")

dict = {'Directory': result_dir, 'Subdirectory': result_sub_dir, 'Filename': result_name,
        'Read_Epoch': result_read_epoch, 'Read_Station': result_read_station, 'TLE_Epoch': result_tle_epoch,
        'Flag': result_flag, 'Events': result_events, 'Event_Times': result_event_times,
        'Station_Alt': result_alt, 'Station_Distance': result_distance,
        'Sat_Lat': result_lat, 'Sat_Lon': result_lon, 'Sat_Height': result_height} 
        
        
df = pd.DataFrame(dict)
     
# saving the dataframe
df.to_csv('L:/DATA/ISIS/Orbitcheck/ISIS-1_orbitcheck.csv')

45714
Procsesing Complete
